In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
import joblib
import os

In [3]:
dx=pd.read_csv('/content/real_legitimate_v1.csv')

In [4]:
dy=pd.read_csv('/content/real_malware_v1.csv')

In [5]:
df = pd.concat([dx, dy], ignore_index=True)
df.to_csv('combined.csv', index=False)

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df=df.to_csv('shuffled_dataset.csv', index=False)

In [7]:
dr=pd.read_csv('/content/shuffled_dataset.csv')

In [8]:
data={
    'Package':dr['Package'][:5000],
    'Malware':dr['Malware'][:5000],
    'prctl':dr['prctl'][:5000],
    'sha256':dr['sha256'][:5000],
    'EarliestModDate':dr['EarliestModDate'][:5000],
    'Malware':dr['Malware'][:5000],
    'write': dr['write'][:5000],
    'read':dr['read'][:5000],
    'write':dr['write'][:5000],
    'munmap':dr['munmap'][:5000],
    'mprotect':dr['mprotect'][:5000],
    'madvise':dr['madvise'][:5000],
    'ioctl':dr['ioctl'][:5000],
    'writev':dr['writev'][:5000],
    'dup':dr['dup'][:5000],
    'lseek':dr['lseek'][:5000],
    'mmap2':dr['mmap2'][:5000],
    'fstat64':dr['fstat64'][:5000],
    'rt_sigprocmask':dr['rt_sigprocmask'][:5000],
   'recvfrom':dr['recvfrom'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'lseek':dr['lseek'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'futex':dr['futex'][:5000],
    'pread':dr['pread'][:5000],
     'FilesInsideAPK':dr['FilesInsideAPK'][:5000],
   'TimesSubmitted':dr['TimesSubmitted'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'Activities':dr['Activities'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'NrIntActivities':dr['NrIntActivities'][:5000],
    'NrIntActivitiesActions':dr['NrIntActivitiesActions'][:5000],
    'TotalIntentFilters':dr['TotalIntentFilters'][:5000],
    'Scanners':dr['Scanners'][:5000],
    'MalFamily':dr['MalFamily'][:5000]
    }
dg=pd.DataFrame(data)
dg

,Package,Malware,prctl,sha256,EarliestModDate,write,read,munmap,mprotect,madvise,...,futex,pread,FilesInsideAPK,TimesSubmitted,Activities,NrIntActivities,NrIntActivitiesActions,TotalIntentFilters,Scanners,MalFamily
0,com.wh.fuelprices\n,0,53,f5d4de00d353b6e432870947aaf945700df7bb97849825...,4/30/2011,6,9,24,33,26,...,45.0,32.0,38.0,5.0,3,1,1,1,67.0,NaN
1,com.lithium.leona.openstud\n,0,63,e344c018bc5cf90d031e16585e98902cbc1836d1f47939...,1/1/1980,34,9,99,588,40,...,126.0,146.0,1546.0,6.0,23,2,3,4,63.0,NaN
2,com.tx.gold\n,1,224,ffb1292bd6f11f2a4d7b8a43fe23f4ad777d3c26b4d2a3...,6/30/2011,1848,1507,145,1274,95,...,530.0,156.0,29.0,19.0,2,1,1,1,66.0,FakeInst
3,com.nqmobile.live.base\n,1,107,33b3b8821c1f2b955dc7607714c7cda375816c3a4174f7...,9/13/2016,6,12,50,605,59,...,136.0,48.0,375.0,1677.0,17,8,9,11,65.0,OpFake
4,com.trz.ysntbx\n,1,202,d27c3adafb7aea3d58d6080819b84dc72fc21421356c52...,7/1/2017,85,87,69,851,77,...,325.0,78.0,218.0,2.0,14,3,3,13,65.0,SMSreg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,com.x25.apps.FuDian\n,0,113,319f62199af70fa183514ca6c091db8d07668c3764406e...,10/28/2010,7,33,94,609,64,...,101.0,142.0,144.0,4.0,13,1,1,1,66.0,DroidKungFu
4996,ch.almana.android.unibas.perssearch\n,0,30,798d41d291d53f1833bb57083aecb1aae9e2a196ea31e5...,7/14/2011,238,120,42,176,26,...,14.0,9.0,21.0,3.0,3,1,2,1,66.0,NaN
4997,com.bitartist.hkradio\n,0,7,8151229d01d43c9885f32aef57ef249f5e7b1dea26ad16...,1/31/2011,121,61,5,2,6,...,29.0,4.0,43.0,3.0,1,1,1,2,65.0,NaN
4998,com.las.xrzauh\n,1,21,d5792389a21e4b4fd867d971ee65647e8d2be4af6fdb0e...,6/27/2017,27,20,26,86,13,...,47.0,29.0,217.0,3.0,14,3,3,12,65.0,SMSreg


In [9]:
dg.isnull().sum()

Package                      0
Malware                      0
prctl                        0
sha256                       0
EarliestModDate              0
write                        0
read                         0
munmap                       0
mprotect                     0
madvise                      0
ioctl                        0
writev                       0
dup                          0
lseek                        0
mmap2                        0
fstat64                      0
rt_sigprocmask               0
recvfrom                     0
getsockopt                   0
clock_gettime                0
futex                        0
pread                        0
FilesInsideAPK               0
TimesSubmitted               0
Activities                   0
NrIntActivities              0
NrIntActivitiesActions       0
TotalIntentFilters           0
Scanners                     0
MalFamily                 2253
dtype: int64

In [10]:
dg=dg.fillna(0)

In [11]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [12]:
if pd.isnull(dg['EarliestModDate']).any():
    # Handle or fill the missing values if necessary
    dg['EarliestModDate'].fillna('fill_value', inplace=True)  # Replace 'fill_value' with an appropriate value
# Convert the 'EarliestModDate' column to datetime
dg['EarliestModDate'] = pd.to_datetime(dg['EarliestModDate'], errors='coerce')

In [13]:
dg.isnull().sum()

Package                    0
Malware                    0
prctl                      0
sha256                     0
EarliestModDate           96
write                      0
read                       0
munmap                     0
mprotect                   0
madvise                    0
ioctl                      0
writev                     0
dup                        0
lseek                      0
mmap2                      0
fstat64                    0
rt_sigprocmask             0
recvfrom                   0
getsockopt                 0
clock_gettime              0
futex                      0
pread                      0
FilesInsideAPK             0
TimesSubmitted             0
Activities                 0
NrIntActivities            0
NrIntActivitiesActions     0
TotalIntentFilters         0
Scanners                   0
MalFamily                  0
dtype: int64

In [14]:
dg=dg.fillna(0)

In [15]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [16]:
initial_year="2017-12-30"
dg['EarliestModDate']=pd.to_datetime(dg['EarliestModDate'],errors='coerce')

In [17]:
dataTrain=dg[dg['EarliestModDate']<=initial_year]
dateTest=dg[dg['EarliestModDate']>initial_year]

In [18]:
print(dataTrain.shape)

(4764, 30)


In [19]:
print(dateTest.shape)

(140, 30)


In [20]:
dg.dtypes

Package                           object
Malware                            int64
prctl                              int64
sha256                            object
EarliestModDate           datetime64[ns]
write                              int64
read                               int64
munmap                             int64
mprotect                           int64
madvise                            int64
ioctl                              int64
writev                             int64
dup                                int64
lseek                              int64
mmap2                              int64
fstat64                            int64
rt_sigprocmask                   float64
recvfrom                         float64
getsockopt                       float64
clock_gettime                    float64
futex                            float64
pread                            float64
FilesInsideAPK                   float64
TimesSubmitted                   float64
Activities      

In [22]:
!pip install scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-multiflow: filename=scikit_multiflow-0.5.3-cp310-cp310-linux_x86_64.whl size=1281850 sha256=0e735786ff6821c50ddd5308cb94d14a4d11ccad55fa85ada7b6505c9e537ac6
  Stored in directory: /root/.cache/pip/wheels/6e/1b/56/45b17a6cf203d98000a45976cb0dd0c4c3f11960e6a505f231
Successfully built scikit-multiflow


In [24]:
initial_year="2017-12-30"
dg['EarliestModDate'] =  pd.to_datetime(dg['EarliestModDate'],errors='coerce')
dataTrain = dg[dg['EarliestModDate'] <=initial_year]
dataTest = dg[dg['EarliestModDate'] >initial_year]
# get years and month
trainYears = dataTrain.EarliestModDate.dt.year.values
trainMonths = dataTrain.EarliestModDate.dt.month.values
testYears = dataTest.EarliestModDate.dt.year.values
testMonths = dataTest.EarliestModDate.dt.month.values
y_train = np.array(dataTrain["Malware"])
y_test = np.array(dataTest["Malware"])
# remove unused columns from dataTrain
UNUSED_COLUMNS = ["Malware", "sha256", "EarliestModDate"]
for c in UNUSED_COLUMNS:
    del dataTrain[c]
    del dataTest[c]
# get values
X_dataTrain = dataTrain.values
X_dataTest = dataTest.values
#feature extraction
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
def feature_extraction(X_dataTrain, X_dataTest):
    from sklearn.feature_extraction.text import TfidfVectorizer
    # initialize X_train and X_test
    X_train = []
    X_test = []
    # save models used
    models = []
    # iterate over each column of X_dataTrain
    for i in range(X_dataTrain.shape[1]):
        # train feature word2vec using column i
        print("Training column {}...".format(i))
        # get train and test data
        train_data = X_dataTrain[:,i]
        test_data = X_dataTest[:,i]
        # initialize and train model
        try:
            tfidf = TfidfVectorizer(max_features=100)
            tfidf.fit(train_data)
            # transform train and test texts to w2v mean
            train_tfidf = tfidf.transform(train_data).todense()
            test_tfidf = tfidf.transform(test_data).todense()
            # if first execution, save only features
            if len(X_train) == 0:
                X_train = train_tfidf
                X_test = test_tfidf
            # concatenate existing features
            else:
                X_train = np.concatenate((X_train, train_tfidf), axis=1)
                X_test = np.concatenate((X_test, test_tfidf), axis=1)
        except:
            pass
        # save model
        models.append(tfidf)
    return(X_train, X_test)
X_train, X_test = feature_extraction(X_dataTrain, X_dataTest)




# normalize features
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def normalization(X_train, X_test):
    # Convert np.matrix to np.array
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    # Replace None values with 0
    X_train[np.isnan(X_train)] = 0
    X_test[np.isnan(X_test)] = 0
    # Perform normalization
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    XX_train = scaler.transform(X_train)
    XX_test = scaler.transform(X_test)
    return XX_train, XX_test
XX_train, XX_test = normalization(X_train, X_test)
# Initialize drift detection method and train base classifier:
# In[8]:
from skmultiflow.drift_detection import DDM, EDDM, ADWIN, PageHinkley
from sklearn.ensemble import RandomForestClassifier
from skmultiflow.meta.adaptive_random_forests import AdaptiveRandomForest as Classifier
#from sklearn.linear_model import SGDClassifier as Classifier
# initialize drift detection method
drift = EDDM()
# create base classifier trained with K samples
# clf = Classifier(random_state=0, drift_detection_method=None, warning_detection_method=None)
clf = Classifier(random_state=0)
# clf2 = Classifier(random_state=0, drift_detection_method=None, warning_detection_method=None)
clf2 = Classifier(random_state=0)
print("Training classifier 1...")
clf.partial_fit(XX_train,y_train, classes=[0,1])
# print("Training classifier 2...")
# clf2.partial_fit(XX_train,y_train, classes=[0,1])
print("Ready for classification.")
# Process data stream:
# In[9]:
from sklearn.metrics import accuracy_score
import math
# drift points
drifts = []
# warning points
warnings = []
# flags for drift and warning
DRIFT = False
WARNING = False
warning_data = []
y_warning = []
# accuracies
accs = []
# predictions
pred = []
true = []
# hits
hits = 0
p = [] # prequential error
s = [] # stdev
n = 1.0 # counter
p.append(1.0) # prequential error
# iterate over stream
for i in range(XX_test.shape[0]):
    # get batch samples
    sample_x = XX_test[i]
    sample_y = y_test[i]
    # predict data
    y_pred = clf.predict([sample_x])
    # save prediction and sample_y
    pred.append(y_pred)
    true.append(sample_y)
    # check if predicted is a hit
    if sample_y == y_pred:
        hits = hits + 1
        p.append(p[-1]-p[-1]/n);
    else:
        p.append(p[-1]+(1-p[-1])/n);
    s.append(math.sqrt(p[-1]*(1-p[-1])/n)) #calc stdev
    n = n+1;
    # feed drift detector
    drift.add_element(sample_y == y_pred)
    # save metrics
    accs.append(float(hits)/float(i+1))
    # detect if warning or drift
    if drift.detected_warning_zone():
        if not WARNING:
            print("Warning in {}...".format(i+1))
            # print("CLF Instances:", clf.instances_seen)
        # fit clf and clf2
        clf.partial_fit([sample_x],[sample_y], classes=[0,1])
        clf2.partial_fit([sample_x],[sample_y], classes=[0,1])
        warnings.append(i+1)
        # save data related to warning...
        warning_data.append(X_dataTest[i])
        y_warning.append(y_test[i])
        # print(X_dataTest.values[stream.sample_idx-1])
        WARNING = True
    elif drift.detected_change():
        print('Change has been detected in {}.'.format(i))
        print(drift.get_info())
        p.append(1.0)
        s.append(0.0);
        n = 1.0
        # save data related to drift (warning yet)
        drifts.append(i)
        # change classifiers
        clf2.partial_fit([sample_x],[sample_y], classes=[0,1])
        clf = clf2
        # clf2 = Classifier(random_state=0, drift_detection_method=None, warning_detection_method=None)
        clf2 = Classifier(random_state=0)
        # print("CLF Instances:", clf.instances_seen)
        # reset drifts information
        drift.reset()
        WARNING = False
        warning_data = []
        y_warning = []
    else:
        WARNING = False
        warning_data = []
        y_warning = []
        # clf2 = Classifier(random_state=0, drift_detection_method=None, warning_detection_method=None)
        clf2 = Classifier(random_state=0)
        clf.partial_fit([sample_x],[sample_y], classes=[0,1])
print("Drifts: ", drifts)
print("AVG Accuracy: ", np.mean(accs))

Training column 0...
Training column 1...
Training column 2...
Training column 3...
Training column 4...
Training column 5...
Training column 6...
Training column 7...
Training column 8...
Training column 9...
Training column 10...
Training column 11...
Training column 12...
Training column 13...
Training column 14...
Training column 15...
Training column 16...
Training column 17...
Training column 18...
Training column 19...
Training column 20...
Training column 21...
Training column 22...
Training column 23...
Training column 24...
Training column 25...
Training column 26...


/usr/local/lib/python3.10/dist-packages/skmultiflow/meta/adaptive_random_forests.py:39: FutureWarning: ’AdaptiveRandomForest’ has been renamed to ‘AdaptiveRandomForestClassifier’ in v0.5.0.
The old name will be removed in v0.7.0
  warnings.warn("’AdaptiveRandomForest’ has been renamed to ‘AdaptiveRandomForestClassifier’ "


Training classifier 1...
Ready for classification.
Warning in 90...
Change has been detected in 129.
EDDM()
Drifts:  [129]
AVG Accuracy:  0.8277663467177533
